In [5]:
from hiive.mdptoolbox.mdp import ValueIteration, PolicyIteration, QLearning
from hiive.mdptoolbox.example import forest
# import hiive_mdptoolbox.example
# import hiive_mdptoolbox
import gym
import numpy as np
import sys
import os
from numpy.random import choice
import pandas as pd
import seaborn as sns
np.random.seed(44)

In [6]:
P, R = forest(S=500, r1=100, r2= 15, p=0.01)

In [7]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [19]:
def test_policy(P, R, policy, test_count=100, gamma=0.9):
    num_state = P.shape[-1]
    total_episode = num_state * test_count
    # start in each state
    total_reward = 0
    for state in range(num_state):
        state_reward = 0
        for state_episode in range(test_count):
            episode_reward = 0
            disc_rate = 1
            while True:
                # take step
                action = policy[state]
                # get next step using P
                probs = P[action][state]
                candidates = list(range(len(P[action][state])))
                next_state =  choice(candidates, 1, p=probs)[0]
                # get the reward
                reward = R[state][action] * disc_rate
                episode_reward += reward
                # when go back to 0 ended
                disc_rate *= gamma
                if next_state == 0:
                    break
            state_reward += episode_reward
        total_reward += state_reward
    return total_reward / total_episode


In [13]:
def trainVI(P, R, discount=0.9, epsilon=[1e-9]):
    vi_df = pd.DataFrame(columns=["Epsilon", "Policy", "Iteration", 
                                  "Time", "Reward", "Value Function"])
    for eps in epsilon:
        vi = ValueIteration(P, R, gamma=discount, epsilon=eps, max_iter=int(1e15))
        vi.run()
        reward = test_policy(P, R, vi.policy)
        info = [float(eps), vi.policy, vi.iter, vi.time, reward, vi.V]
        df_length = len(vi_df)
        vi_df.loc[df_length] = info
    return vi_df

In [14]:
vi_df = trainVI(P, R, epsilon=[1e-1, 1e-3, 1e-6, 1e-9, 1e-12, 1e-15])
vi_df

,Epsilon,Policy,Iteration,Time,Reward,Value Function
0,1.000000e-01,"(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",79,0.069786,2.744159,"(4.710556185449387, 5.239434944489701, 5.23943..."
1,1.000000e-03,"(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",119,0.037132,2.738842,"(4.7117745667154995, 5.240595870281114, 5.2405..."
2,1.000000e-06,"(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",179,0.199475,2.736443,"(4.711792669916437, 5.240613400253226, 5.24061..."
3,1.000000e-09,"(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",239,0.058018,2.726877,"(4.711792702216012, 5.240613431989174, 5.24061..."
4,1.000000e-12,"(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",299,0.387262,2.749473,"(4.711792702273827, 5.240613432046434, 5.24061..."
5,1.000000e-15,"(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",348,0.363754,2.724796,"(4.7117927022739305, 5.240613432046538, 5.2406..."


In [15]:
pi = PolicyIteration(P, R, gamma=0.9, max_iter=1e6)
pi.run()
pi_pol = pi.policy
pi_reward = test_policy(P, R, pi_pol)
pi_iter = pi.iter
pi_time = pi.time
pi_iter, pi_time, pi_reward

(46, 3.1077523231506348, 2.7201021303049737)

# Q-Learning

In [17]:
def trainQ(P, R, discount=0.9, alpha_dec=[0.99], alpha_min=[0.001], 
            epsilon=[1.0], epsilon_decay=[0.99], n_iter=[1000000]):
    q_df = pd.DataFrame(columns=["Iterations", "Alpha Decay", "Alpha Min", 
                                 "Epsilon", "Epsilon Decay", "Reward",
                                 "Time", "Policy", "Value Function",
                                 "Training Rewards"])
    
    count = 0
    for i in n_iter:
        for eps in epsilon:
            for eps_dec in epsilon_decay:
                for a_dec in alpha_dec:
                    for a_min in alpha_min:
                        q = QLearning(P, R, discount, alpha_decay=a_dec, 
                                      alpha_min=a_min, epsilon=eps, 
                                      epsilon_decay=eps_dec, n_iter=i)
                        q.run()
                        reward = test_policy(P, R, q.policy)
                        count += 1
                        print("{}: {}".format(count, reward))
                        st = q.run_stats
                        rews = [s['Reward'] for s in st]
                        info = [i, a_dec, a_min, eps, eps_dec, reward, 
                                q.time, q.policy, q.V, rews]
                        
                        df_length = len(q_df)
                        q_df.loc[df_length] = info
    return q_df

In [20]:
alpha_decs = [0.99, 0.999]
alpha_mins =[0.001, 0.0001]
eps = [10.0, 1.0]
eps_dec = [0.99, 0.999]
iters = [1000000, 10000000]
q_df = trainQ(P, R, discount=0.9, alpha_dec=alpha_decs, alpha_min=alpha_mins, 
            epsilon=eps, epsilon_decay=eps_dec, n_iter=iters)

1: 0.802
2: 2.5774950452236354
3: 2.631858509961703
4: 2.646175901770941
5: 2.539573504201036
6: 2.6069021701575914
7: 2.5693660624419152
8: 2.6374556489936967
9: 2.6703789853483073
10: 2.690222376440896
11: 2.6191894698646014
12: 2.6511543095675845
13: 0.82
14: 2.679801290177299
15: 2.6501418339752325
16: 2.707013345485346
17: 2.723851173383091
18: 1.026
19: 2.7562338681255514
20: 2.8183151291782824
21: 2.7079361673822815
22: 2.8350224573241065
23: 2.7398968958398227
24: 2.788664469001925
25: 2.802476203667823
26: 2.8659148404404533
27: 2.6514897951414356
28: 2.761586387754335
29: 2.762742027318329
30: 2.851093916926482
31: 2.7646190558922
32: 2.8128115123936395


In [23]:
q_df

,Iterations,Alpha Decay,Alpha Min,Epsilon,Epsilon Decay,Reward,Time,Policy,Value Function,Training Rewards
0,1000000,0.990,0.0010,10.0,0.990,0.802000,59.492043,"(0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, ...","(4.713041412297211, 5.241136060452282, 5.23995...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1000000,0.990,0.0001,10.0,0.990,2.577495,68.662860,"(0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, ...","(4.6737963546775925, 5.2021362248749154, 4.381...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ..."
2,1000000,0.999,0.0010,10.0,0.990,2.631859,59.798015,"(0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, ...","(4.711724251799691, 5.239457021467696, 5.23972...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
3,1000000,0.999,0.0001,10.0,0.990,2.646176,53.276327,"(0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, ...","(4.7099275467466715, 5.238704855970898, 5.0982...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
4,1000000,0.990,0.0010,10.0,0.999,2.539574,55.888296,"(0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, ...","(4.708855253748645, 5.2378874490337415, 5.2396...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, ..."
5,1000000,0.990,0.0001,10.0,0.999,2.606902,56.844924,"(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...","(4.667569232222921, 5.196535559438114, 4.33331...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
6,1000000,0.999,0.0010,10.0,0.999,2.569366,55.906310,"(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(4.711215655937732, 5.242053489773692, 5.24208...","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."
7,1000000,0.999,0.0001,10.0,0.999,2.637456,54.514348,"(0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, ...","(4.709364929082401, 5.238125417620886, 5.12826...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
8,1000000,0.990,0.0010,1.0,0.990,2.670379,56.121849,"(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...","(4.710426417260209, 5.239160740655902, 5.24024...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,1000000,0.990,0.0001,1.0,0.990,2.690222,58.525208,"(0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, ...","(4.668657569840826, 5.197289517016076, 4.32268...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [30]:
pi_pol == q_df.Policy

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
Name: Policy, dtype: bool

In [24]:
q_df.groupby("Iterations").mean()

,Alpha Decay,Alpha Min,Epsilon,Epsilon Decay,Reward,Time
Iterations,,,,,,
1000000,0.9945,0.00055,5.5,0.9945,2.406171,57.976636
10000000,0.9945,0.00055,5.5,0.9945,2.666791,513.044849


In [25]:
q_df.groupby("Epsilon Decay").mean()

,Alpha Decay,Alpha Min,Epsilon,Reward,Time
Epsilon Decay,,,,,
0.990,0.9945,0.00055,5.5,2.480896,293.982958
0.999,0.9945,0.00055,5.5,2.592065,277.038527


In [26]:
q_df.groupby("Alpha Decay").mean()

,Alpha Min,Epsilon,Epsilon Decay,Reward,Time
Alpha Decay,,,,,
0.990,0.00055,5.5,0.9945,2.372588,286.648156
0.999,0.00055,5.5,0.9945,2.700373,284.373330


In [31]:
q_df.groupby("Epsilon Decay").mean()


,Alpha Decay,Alpha Min,Epsilon,Reward,Time
Epsilon Decay,,,,,
0.990,0.9945,0.00055,5.5,2.480896,293.982958
0.999,0.9945,0.00055,5.5,2.592065,277.038527


In [32]:
q_df.groupby("Alpha Min").mean()

,Alpha Decay,Epsilon,Epsilon Decay,Reward,Time
Alpha Min,,,,,
0.0001,0.9945,5.5,0.9945,2.622227,284.723311
0.0010,0.9945,5.5,0.9945,2.450735,286.298174
